In [1]:
from bs4 import BeautifulSoup
import requests

from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

import pandas as pd
import numpy as np
import string, os 

import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

HEADERS = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
SITE_LINK = 'http://www.romanianvoice.com/poezii/poeti/eminescu.php' #where i am getting the poems from
ROOT_LINK = 'http://www.romanianvoice.com/poezii' #used later when constructing the link for the poem extractor
SPLIT_KEYWORD = 'Mihai Eminescu' #keyword to substring the poems


In [2]:
def get_poem_links(site_link):
    poem_links = []
    req = requests.get(site_link, headers=HEADERS)
    soup = BeautifulSoup(req.content, 'html.parser')
    for a in soup.find_all('a', href=True):
        
        poem_links.append(a['href'].replace('..', ROOT_LINK)) #getting from ../poezii/xxx to a working link
    
    return poem_links[2:] #first 2 links lead to pages we don't need
        
poem_links = get_poem_links(SITE_LINK)
print(poem_links[:5])

['http://www.romanianvoice.com/poezii/poezii/adio.php', 'http://www.romanianvoice.com/poezii/poezii/afarai.php', 'http://www.romanianvoice.com/poezii/poezii/marmure.php', 'http://www.romanianvoice.com/poezii/poezii/frageda.php', 'http://www.romanianvoice.com/poezii/poezii/calin.php']


In [3]:
from tqdm import tqdm

def get_poem(link):
    req = requests.get(link, headers=HEADERS)
    soup = BeautifulSoup(req.content, 'html.parser')
    text = soup.get_text()
    
    split_text = text.split('\n')
            
    return split_text[47:]

poem_list = []

for poem_link in tqdm(poem_links):
    try:
        #print(poem_link)
        poem_list.extend(get_poem(poem_link))
    except:
        pass


100%|██████████| 95/95 [01:06<00:00,  1.42it/s]


In [4]:
verses_list = [verse for verse in poem_list if verse != '']
print(len(verses_list))
print(verses_list[:10])

4666
['De-acuma nu te-oi mai vedea,', 'Rămâi, rămâi, cu bine!', 'Mă voi feri în calea mea', 'De tine.', 'De astăzi dar tu fă ce vrei,', 'De astăzi nu-mi mai pasă', 'Că cea mai dulce-ntre femei', 'Mă lasă.', 'Căci nu mai am de obicei', 'Ca-n zilele acele,']


In [5]:
import string

def clean_text(text):
    text = text.replace('-n',' in').replace('-mi',' imi')
    puncts = [',','.','?','!']
    #text = text.replace('ă','a').replace('â','a').replace('î','i').replace('ș','s').replace('ț','t').replace('ş','')
    text = "".join(v for v in text if v not in puncts).lower()
    
    return text
  
corpus = [clean_text(x) for x in verses_list]      
print(corpus[:10])  
  

['de-acuma nu te-oi mai vedea', 'rămâi rămâi cu bine', 'mă voi feri în calea mea', 'de tine', 'de astăzi dar tu fă ce vrei', 'de astăzi nu imi mai pasă', 'că cea mai dulce intre femei', 'mă lasă', 'căci nu mai am de obicei', 'ca in zilele acele']


In [6]:
tokenizer = Tokenizer()

def get_token_sequence(corpus):
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index)+1
    
    input_sequences = []
    
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
            
    return input_sequences, total_words

inp_sequences, total_words = get_token_sequence(corpus)
inp_sequences[:10]



[[1, 338],
 [1, 338, 16],
 [1, 338, 16, 19],
 [1, 338, 16, 19, 168],
 [1, 338, 16, 19, 168, 22],
 [1, 338, 16, 19, 168, 22, 375],
 [264, 264],
 [264, 264, 5],
 [264, 264, 5, 105],
 [35, 53]]

In [7]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)
 
print(predictors[0])
print(max_sequence_len)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
17


In [8]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    model.add(LSTM(100, return_sequences=True))
    model.add(Dropout(0.3))
    
    model.add(LSTM(100))
    model.add(Dropout(0.3))
    
    model.add(Dense(total_words, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

print(total_words)

model = create_model(max_sequence_len, total_words)
model.summary()


6713
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 16, 10)            67130     
_________________________________________________________________
lstm (LSTM)                  (None, 16, 100)           44400     
_________________________________________________________________
dropout (Dropout)            (None, 16, 100)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 6713)              678013    
Total params: 869,943
Trainable params: 869,943
Non-trainable params: 0
_____________________________________________

In [13]:
history = model.fit(predictors, label, epochs=100)

Epoch 1/100
835/835 [==============================] - 4s 5ms/step - loss: 5.8536
Epoch 2/100
835/835 [==============================] - 4s 5ms/step - loss: 5.7808
Epoch 3/100
835/835 [==============================] - 4s 5ms/step - loss: 5.7084
Epoch 4/100
835/835 [==============================] - 4s 5ms/step - loss: 5.6277
Epoch 5/100
835/835 [==============================] - 4s 5ms/step - loss: 5.5569
Epoch 6/100
835/835 [==============================] - 4s 5ms/step - loss: 5.4859
Epoch 7/100
835/835 [==============================] - 4s 5ms/step - loss: 5.4225
Epoch 8/100
835/835 [==============================] - 4s 5ms/step - loss: 5.3463
Epoch 9/100
835/835 [==============================] - 4s 5ms/step - loss: 5.2843
Epoch 10/100
835/835 [==============================] - 4s 5ms/step - loss: 5.2218
Epoch 11/100
835/835 [==============================] - 4s 5ms/step - loss: 5.1598
Epoch 12/100
835/835 [==============================] - 4s 5ms/step - loss: 5.0923
Epoch 13/100


835/835 [==============================] - 4s 5ms/step - loss: 3.1835
Epoch 100/100
835/835 [==============================] - 4s 5ms/step - loss: 3.1637


In [14]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        print(predicted)
        
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text

In [17]:
generate_text(seed_text = "eu", next_words = 10,model = model,max_sequence_len = max_sequence_len)

[16]
[22]
[841]
[1]
[182]
[24]
[16]
[22]
[255]
[1315]


'eu nu mai şti de mine că nu mai plânge cuvântul'

In [59]:
sum = 200
for i in range(30):
    sum+=sum/10
    
print(sum)

3489.8804537772817
